In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Imports section
import numpy as np
import os
from matplotlib import pyplot as plt
import pickle
import cv2
import time
import tensorflow as tf

In [ ]:
# Set number of parallel processing units for dataset reading
units=tf.data.experimental.AUTOTUNE

class Dataset_Generator():
  def __init__(self, img_size, batch_size=64):
    # Pathnames
    dir="/content/drive/My Drive/StackGAN/Dataset/CUB_200_2011"
    train_dir=dir+"/train"
    test_dir=dir+"/test"
    train_embedpaths=train_dir+"/char-CNN-RNN-embeddings.pickle"
    test_embedpaths=test_dir+"/char-CNN-RNN-embeddings.pickle"
    train_filepaths=train_dir+"/filenames.pickle"
    test_filepaths=test_dir+"/filenames.pickle"
    bboxes_paths=dir+"/bounding_boxes.txt"
    image_paths =dir+"/images.txt"
    images=dir+"/images"

    # Image dimensions and batch sizes
    self.img_h=img_size[0]
    self.img_w=img_size[1]
    self.batch_size=batch_size

    # Generate train image filenames
    with open(train_filepaths, 'rb') as f:
      self.train_filenames=pickle.load(f, encoding='latin1')
      self.train_filenames=[os.path.join(images, fname)+'.jpg' for fname in self.train_filenames]

    # Generate test image filenames
    with open(test_filepaths, 'rb') as f:
      self.test_filenames=pickle.load(f, encoding='latin1')
      self.test_filenames=[os.path.join(images, fname)+'.jpg' for fname in self.test_filenames]

    # Generate train Char-CNN-RNN embedding filenames
    with open(train_embedpaths, 'rb') as f:
      self.train_embeds=pickle.load(f, encoding='latin1')

    # Generate test Char-CNN-RNN embedding filenames
    with open(test_embedpaths, 'rb') as f:
      self.test_embeds=pickle.load(f, encoding='latin1')

    # Generate bounding boxes within a dictionary, reading them from the file
    bounding_boxes={}
    with open(bboxes_paths, 'rb') as f:
      coords=f.read()
      coords=coords.splitlines()
      coords=[coord.decode('utf-8') for coord in coords]
      for i in range(len(coords)):
        bounding_box=coords[i].split()
        bounding_boxes[bounding_box[0]]=[int(float(c)) for c in coords[i].split()][1:]

    # Generate image filenames within a dictionary, read from the file
    image_fnames={}
    with open(image_paths, 'rb') as f:
      im_fs=f.read()
      im_fs=im_fs.splitlines()
      im_fs=[im_f.decode('utf-8') for im_f in im_fs]
      for i in range(len(im_fs)):
        im_f=im_fs[i].split()
        image_fnames[im_f[0]]=im_f[1]

    # Mapping generated bounding boxes with their corresponding images
    bboxes_map={}
    for im_f in bounding_boxes.keys():
      bboxes_map[images+'/'+image_fnames[im_f]]=bounding_boxes[im_f]
    # Generate train bounding boxes
    self.train_bboxes=[]
    for i in range(len(self.train_filenames)):
      self.train_bboxes.append(bboxes_map[self.train_filenames[i]])

    # Generate test bounding boxes
    self.test_bboxes=[]
    for i in range(len(self.test_filenames)):
      self.test_bboxes.append(bboxes_map[self.test_filenames[i]])

  # Crop images to bounding boxes
  def crop(self, image, box):
    image=image.numpy()
    if box is not None:
      x, y, w, h=box # Unpack to get bounding box co-ordinates
      image=image[y:(y+h), x:(x+w)]
      image=cv2.resize(image, (self.img_w, self.img_h))
    return image

  # Read image from the dataset
  def read_image(self, path, embeds, box):
    image=tf.io.read_file(path)
    image=tf.image.decode_jpeg(image, channels=3)
    image=tf.py_function(func=self.crop, inp=[image, box], Tout=tf.float32)
    image.set_shape([self.img_w, self.img_h, 3])
    # Normalizing values reduces dataset parse duration
    image=(image-127.5)/127.5 # Normalizing image pixel values down
    embed_no=np.random.randint(0, embeds.shape[0]-1)
    embed=embeds[embed_no]
    return image, embed

  # Get training dataset
  def get_train_dataset(self):
    size=len(self.train_filenames)
    dataset=tf.data.Dataset.from_tensor_slices((self.train_filenames, self.train_embeds, self.train_bboxes))
    dataset=dataset.shuffle(size)
    dataset=dataset.repeat()
    dataset=dataset.map(self.read_image, num_parallel_calls=units)
    dataset=dataset.batch(self.batch_size, drop_remainder=True)
    dataset=dataset.prefetch(1)
    return dataset

  # Get testing dataset
  def get_test_dataset(self):
    size=len(self.train_filenames)
    dataset=tf.data.Dataset.from_tensor_slices((self.test_filenames, self.test_embeds, self.test_bboxes))
    dataset=dataset.shuffle(size)
    dataset=dataset.repeat()
    dataset=dataset.map(self.read_image, num_parallel_calls=units)
    dataset=dataset.batch(self.batch_size, drop_remainder=True)
    dataset=dataset.prefetch(1)
    return dataset

In [ ]:
start=time.time()
dataset=Dataset_Generator(img_size=(64, 64))
train=dataset.get_train_dataset()
test=dataset.get_test_dataset()
print("Time elapsed:",time.time()-start,"seconds")

Time elapsed: 44.00000715255737 seconds


In [ ]:
batch_iter=iter(train)
img_batch, _=next(batch_iter)